# Uno Card Detection

This code is an implementation of a computer vision algorithm for detecting Uno cards in a video stream and on individual images using Python and OpenCV. 

The algorithm first extracts features and color information from each frame of the video stream, and then uses a pre-trained Random Forest Classifier to predict the type of the card. 

The predicted card type is displayed on the video stream in real-time using OpenCV. 

The code allows for the detection of specific types of Uno cards: "Number Cards", "Pick Two," "Reverse," and "Skip" 

The code can be easily adapted to detect other types of cards or objects by training a new classifier on a diverse and large dataset.

Below are the blocks of code that I used to build this project.

### Color Extraction
For extracting the color of the card we first select the region of onterest and crop the rest of the image. 

After that I have used pre-defined HSV ranges for blue, red, green and yellow color to create a bitwise and mask to select specific colors in the card. 
After applying the mask I check for the maximum pixels of the color that remained after applying the mask and then classify the color of the card. 


In [50]:
import cv2
import numpy as np   
 # Extracting color of the card
img_colour = cv2.imread('./unocards/ys.jpg')   # open the saved image in colour

img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W
img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV) # Convert to HSV
img_sm = cv2.blur(img, (7, 7))         # smoothing
thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation

kernel = np.ones((3, 3), np.uint8)
img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny

# Select the region of interest i.e the card and crop the background 
contour1 = contours[1]
x, y, w, h = cv2.boundingRect(contour1) # Find the bounding rectangle of contour 1

crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle

# Checking for color using HSV ranges

# Convert the image to HSV
hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

# Define the range of colors for the Uno card

# Green color range
green_lower = np.array([36, 25, 25])
green_upper = np.array([65, 255, 255])
# Yellow color range
y_lower = np.array([15, 50, 50])  
y_upper = np.array([30, 255, 255])

# Blue color range
blue_lower = np.array([100, 150, 150])
blue_upper = np.array([130, 255, 255])

# Red color range (two ranges to account for hue wraparound)
red_lower1 = np.array([0, 50, 50])
red_upper1 = np.array([10, 255, 255])

# Create a mask for the red color
mask_yellow = cv2.inRange(hsv, y_lower, y_upper)
mask_red = cv2.inRange(hsv, red_lower1, red_upper1)
mask_green = cv2.inRange(hsv, green_lower, green_upper)
mask_blue = cv2.inRange(hsv, blue_lower, blue_upper)
# Bitwise-AND mask and original image
res_y = cv2.bitwise_and(crop_img,crop_img, mask= mask_yellow)
res_r = cv2.bitwise_and(crop_img,crop_img, mask= mask_red)
res_g = cv2.bitwise_and(crop_img,crop_img, mask= mask_green)
res_b = cv2.bitwise_and(crop_img,crop_img, mask= mask_blue)

# Check if the card is red, yellow , blue or green
counts = [np.count_nonzero(res_r), np.count_nonzero(res_y), np.count_nonzero(res_g), np.count_nonzero(res_b)]
colors = [0,1,2,3] # Red = 0, yellow = 1, green = 2, blue = 3

if max(counts) == 0:
    print("Card Color not recognized")
else:
    color_index = counts.index(max(counts))
    color = colors[color_index]

print(color)

1


### Extracting Features and creating a Dataset of features

In this code I get the contours of the card and select a particular contour that I am interested in i.e the contour of the number or the type of the card inside the ellipse. 

I select the contour by using the heirarchy retrived by the RETR_TREE method in OpenCV. The images I have used in this project only contain the uno card on a black background. If there is any noise in the images the features extractin will fail as it'll be difficult to reach to the contour of the number of the card inside the ellipse. 

After selecting all the child contours of the ellipse I merge them into a single conotur and apply feature extraction operations on it. 

The feature that I extract from the merged contour are: Aspect Ratio, Extent, Solidity, Equivalent Diameter, Hu Moments and the number of holes.

After extracting the contours I loop through all the images in the dataset, extract the features from each image and save the features with the card type as the classes using joblib. 

The dataset is similar to the iris dataset in structure i.e it contains a dictionary of arrays of features and classes. 

In [200]:
import cv2
import numpy as np
import joblib
import os
import re

def feature_extract(image_path):

    img_colour = cv2.imread(image_path)

    img_gray = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)

    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # Create a list to store the areas of each contour
    areas = []

    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)

    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))

    # Select the third largest contour and its children
    third_largest_contour_index = sorted_areas_indices[2]

    # Define a recursive function to find all the children of a given contour index
    def find_children_indices(hierarchy, index):
        children_indices = []
        child_index = hierarchy[0][index][2]
        while child_index != -1:
            children_indices.append(child_index)
            grandchild_indices = find_children_indices(hierarchy, child_index)
            children_indices.extend(grandchild_indices)
            child_index = hierarchy[0][child_index][0]
        return children_indices

    children_indices = find_children_indices(hierarchy, third_largest_contour_index)

    # Merge the contours of the children of the third largest contour into a single variable for feature extraction
    merged_contour = None
    for child_index in children_indices:
        if merged_contour is None:
            merged_contour = contours[child_index]
        else:
            merged_contour = np.concatenate((merged_contour, contours[child_index]))


    
    # Aspect Ratio
    x,y,w,h = cv2.boundingRect(merged_contour)
    aspect_ratio = float(w)/h

    # Extent
    contour_area = cv2.contourArea(merged_contour)
    x, y, w, h = cv2.boundingRect(merged_contour)
    rect_area = w*h # Area for the bounding rectangle
    extent = float(contour_area)/rect_area

    # Solidity
    area = cv2.contourArea(merged_contour)
    hull = cv2.convexHull(merged_contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area

    # Equivalent Diameter 
    area = cv2.contourArea(merged_contour)
    equi_diameter = np.sqrt(4*area/np.pi)

    #Hu Moments 
    moments = cv2.moments(merged_contour)
    hu_moments = cv2.HuMoments(moments)

    # Convert the Hu moments into logscale
    hu_moments_log = -1 * cv2.log(abs(hu_moments))

    # Assign the transformed Hu moments to variables
    h1 = hu_moments_log[0][0]
    h2 = hu_moments_log[1][0]
    h3 = hu_moments_log[2][0]
    h4 = hu_moments_log[3][0]
    h5 = hu_moments_log[4][0]
    h6 = hu_moments_log[5][0]
    h7 = hu_moments_log[6][0]

    #Finding number of holes 
    x, y, w, h = cv2.boundingRect(merged_contour) # Find the bounding rectangle of contour 1

    crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle
    img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Create a list to store the areas of each contour
    areas = []

    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)

    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))

    # Select the second largest contour and its children
    num_contour_index = sorted_areas_indices[1]
    second_largest_contour = contours[num_contour_index]
    num_holes = 0

    # Iterate over the hierarchy to count the number of child contours
    if hierarchy.size > 0:
        hierarchy = hierarchy[0]
        current_contour = hierarchy[num_contour_index][2]
        while current_contour != -1:
            num_holes += 1
            current_contour = hierarchy[current_contour][0]

    return [aspect_ratio, extent, solidity, equi_diameter, num_holes, h1, h2, h3, h4, h5, h6, h7]


# Create empty lists to store the six features and labels
features = []
labels = []
images_folder = './unocards/'
# Loop through each image and extract the six features from its contour
for image_file in os.listdir(images_folder):
    image_path = os.path.join(images_folder, image_file)
    feature_set = feature_extract(image_path)
    features.append(feature_set)
    match = re.search(r'\d+', image_file)
    if match:
        file_num = int(match.group())
    labels.append(file_num)
features = np.array(features)
labels = np.array(labels)
    # labels = np.append(labels, int(os.path.splitext(image_file[1:])[0]))
joblib.dump([{'data': features, 'target' : labels}], 'dataset_final.joblib')

['dataset_final.joblib']

##### Loading the dataset

In [9]:
unodata = joblib.load("dataset_final.joblib")
# print(unodata)

## Training Random Forest Classifier on the created Dataset

I have used Random Forest Classifier for this problem because it performs really well on the problems with multiple classes. 

In [202]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X = unodata[0]['data']
y = unodata[0]['target']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)
joblib.dump(rfc, "uno_rfc.joblib")    # save the model after training, remember the file name

['uno_rfc.joblib']

##### Got an accuracy on 90% on the dataset when performed a 80 - 20 Train - Test Split. 

In [203]:
rfc = joblib.load("uno_rfc.joblib")    # load the model to test it
score = rfc.score(X_test, y_test)
print(score)

0.9


## Testing the trained model.

In [3]:
import cv2
import numpy as np
import joblib

def feature_extract(image_path):
    # Image Pre-processing
    img_colour = cv2.imread(image_path)

    img_gray = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)

    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # Create a list to store the areas of each contour
    areas = []

    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)

    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))

    # Select the third largest contour and its children
    third_largest_contour_index = sorted_areas_indices[2]

    # Define a recursive function to find all the children of a given contour index
    def find_children_indices(hierarchy, index):
        children_indices = []
        child_index = hierarchy[0][index][2]
        while child_index != -1:
            children_indices.append(child_index)
            grandchild_indices = find_children_indices(hierarchy, child_index)
            children_indices.extend(grandchild_indices)
            child_index = hierarchy[0][child_index][0]
        return children_indices

    children_indices = find_children_indices(hierarchy, third_largest_contour_index)

    # Merge the contours of the children of the third largest contour into a single variable for feature extraction
    merged_contour = None
    for child_index in children_indices:
        if merged_contour is None:
            merged_contour = contours[child_index]
        else:
            merged_contour = np.concatenate((merged_contour, contours[child_index]))


    
    # Aspect Ratio
    x,y,w,h = cv2.boundingRect(merged_contour)
    aspect_ratio = float(w)/h

    # Extent
    contour_area = cv2.contourArea(merged_contour)
    x, y, w, h = cv2.boundingRect(merged_contour)
    rect_area = w*h # Area for the bounding rectangle
    extent = float(contour_area)/rect_area

    # Solidity
    area = cv2.contourArea(merged_contour)
    hull = cv2.convexHull(merged_contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area

    # Equivalent Diameter 
    area = cv2.contourArea(merged_contour)
    equi_diameter = np.sqrt(4*area/np.pi)

    #Hu Moments 
    moments = cv2.moments(merged_contour)
    hu_moments = cv2.HuMoments(moments)

    # Convert the Hu moments into logscale
    hu_moments_log = -1 * cv2.log(abs(hu_moments))

    # Assign the transformed Hu moments to variables
    h1 = hu_moments_log[0][0]
    h2 = hu_moments_log[1][0]
    h3 = hu_moments_log[2][0]
    h4 = hu_moments_log[3][0]
    h5 = hu_moments_log[4][0]
    h6 = hu_moments_log[5][0]
    h7 = hu_moments_log[6][0]

    #Finding number of holes 
    x, y, w, h = cv2.boundingRect(merged_contour) # Find the bounding rectangle of contour 1

    crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle
    img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Create a list to store the areas of each contour
    areas = []

    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)

    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))

    # Select the second largest contour and its children
    num_contour_index = sorted_areas_indices[1]
    second_largest_contour = contours[num_contour_index]
    num_holes = 0

    # Iterate over the hierarchy to count the number of child contours
    if hierarchy.size > 0:
        hierarchy = hierarchy[0]
        current_contour = hierarchy[num_contour_index][2]
        while current_contour != -1:
            num_holes += 1
            current_contour = hierarchy[current_contour][0]
    return [aspect_ratio, extent, solidity, equi_diameter, num_holes, h1, h2, h3, h4, h5, h6, h7]

def color_extract(image_path):
    img_colour = cv2.imread(image_path)   # open the saved image in colour

    # Image Pre-processing
    img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W
    img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV) # Convert to HSV
    img_sm = cv2.blur(img, (7, 7))         # smoothing
    thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
    # img_th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 35, 2)
    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
    img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny

    # Select the region of interest i.e the card and crop the background 
    contour1 = contours[1]
    x, y, w, h = cv2.boundingRect(contour1) # Find the bounding rectangle of contour 1

    crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle

    # Checking for color using HSV ranges

    # Convert the image to HSV
    hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

    # Define the range of colors for the Uno card

    # Green color range
    green_lower = np.array([36, 25, 25])
    green_upper = np.array([65, 255, 255])
    green_range = np.array([green_lower, green_upper])

    # Yellow color range
    y_lower = np.array([15, 50, 50])  
    y_upper = np.array([30, 255, 255])
    yellow_range = np.array([y_lower, y_upper])

    # Blue color range
    blue_lower = np.array([100, 150, 150])
    blue_upper = np.array([130, 255, 255])
    blue_range = np.array([blue_lower, blue_upper])

    # Red color range (two ranges to account for hue wraparound)
    red_lower1 = np.array([0, 50, 50])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([170, 50, 50])
    red_upper2 = np.array([180, 255, 255])
    red_range = np.array([red_lower1, red_upper1, red_lower2, red_upper2])

    # Create a mask for the red color
    mask_yellow = cv2.inRange(hsv, y_lower, y_upper)
    mask_red = cv2.inRange(hsv, red_lower1, red_upper1)
    mask_green = cv2.inRange(hsv, green_lower, green_upper)
    mask_blue = cv2.inRange(hsv, blue_lower, blue_upper)
    # Bitwise-AND mask and original image
    res_y = cv2.bitwise_and(crop_img,crop_img, mask= mask_yellow)
    res_r = cv2.bitwise_and(crop_img,crop_img, mask= mask_red)
    res_g = cv2.bitwise_and(crop_img,crop_img, mask= mask_green)
    res_b = cv2.bitwise_and(crop_img,crop_img, mask= mask_blue)

    # Check if the card is red, yellow , blue or green
    counts = [np.count_nonzero(res_r), np.count_nonzero(res_y), np.count_nonzero(res_g), np.count_nonzero(res_b)]
    colors = ["RED", "YELLOW", "GREEN", "BLUE"]

    if max(counts) == 0:
        print("Card Color not recognized")
    else:
        color_index = counts.index(max(counts))
        det_color = colors[color_index]
    return det_color

## Testing on Images

In [4]:
feat = []
image_path = './uno_images_test/y0.jpg'
fx = feature_extract(image_path)
col = color_extract(image_path)
feat.append(fx)
feat = np.array(feat)
rfc = joblib.load("uno_rfc.joblib") 
predict = rfc.predict(feat)
if predict == np.array([100]): # Pick two card is labeled as 100 in the dataset
    predict = 'Pick Two'
elif predict == np.array([200]): # Reverse card is labeled as 100 in the dataset
    predict = 'Reverse'
elif predict == np.array([300]): # Skip card is labeled as 100 in the dataset
    predict = 'Skip'
print(col, predict)
# print(type(predict))

YELLOW [0]


## Testing on Camera stream

In [1]:
import cv2
import numpy as np
import joblib

def feature_extract(frame):
    img_colour = frame
    img_gray = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)
    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Create a list to store the areas of each contour
    areas = []
    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)
    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))
    # Select the third largest contour and its children
    third_largest_contour_index = sorted_areas_indices[2]
    # Define a recursive function to find all the children of a given contour index
    def find_children_indices(hierarchy, index):
        children_indices = []
        child_index = hierarchy[0][index][2]
        while child_index != -1:
            children_indices.append(child_index)
            grandchild_indices = find_children_indices(hierarchy, child_index)
            children_indices.extend(grandchild_indices)
            child_index = hierarchy[0][child_index][0]
        return children_indices
    children_indices = find_children_indices(hierarchy, third_largest_contour_index)
    # Merge the contours of the children of the third largest contour into a single variable for feature extraction
    merged_contour = None
    for child_index in children_indices:
        if merged_contour is None:
            merged_contour = contours[child_index]
        else:
            merged_contour = np.concatenate((merged_contour, contours[child_index]))
    # Aspect Ratio
    x,y,w,h = cv2.boundingRect(merged_contour)  
    aspect_ratio = float(w)/h
    # Extent
    contour_area = cv2.contourArea(merged_contour)
    x, y, w, h = cv2.boundingRect(merged_contour)
    rect_area = w*h # Area for the bounding rectangle
    extent = float(contour_area)/rect_area
    # Solidity
    area = cv2.contourArea(merged_contour)
    hull = cv2.convexHull(merged_contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area
    # Equivalent Diameter 
    area = cv2.contourArea(merged_contour)
    equi_diameter = np.sqrt(4*area/np.pi)
    #Hu Moments 
    moments = cv2.moments(merged_contour)
    hu_moments = cv2.HuMoments(moments)
    # Convert the Hu moments into logscale
    hu_moments_log = -1 * cv2.log(abs(hu_moments))
    # Assign the transformed Hu moments to variables
    h1 = hu_moments_log[0][0]
    h2 = hu_moments_log[1][0]
    h3 = hu_moments_log[2][0]
    h4 = hu_moments_log[3][0]
    h5 = hu_moments_log[4][0]
    h6 = hu_moments_log[5][0]
    h7 = hu_moments_log[6][0]

    #Finding number of holes 
    x, y, w, h = cv2.boundingRect(merged_contour) # Find the bounding rectangle of contour 1
        
    crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle
            
    img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    _, img_th = cv2.threshold(img_gray, 160, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Create a list to store the areas of each contour
    areas = []

    # Iterate over the contours and calculate the area of each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)

    # Use the numpy.argsort() function to get the indices of the areas in descending order
    sorted_areas_indices = np.argsort(-np.array(areas))

    # Select the second largest contour and its children
    num_contour_index = sorted_areas_indices[1]
    second_largest_contour = contours[num_contour_index]
    num_holes = 0

    # Iterate over the hierarchy to count the number of child contours
    if hierarchy.size > 0:
        hierarchy = hierarchy[0]
        current_contour = hierarchy[num_contour_index][2]
        while current_contour != -1:
            num_holes += 1
            current_contour = hierarchy[current_contour][0]
    return [aspect_ratio, extent, solidity, equi_diameter, num_holes, h1, h2, h3, h4, h5, h6, h7]
       

def color_extract(frame):
    img_colour = frame

    # Image Pre-processing
    img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W
    img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV) # Convert to HSV
    img_sm = cv2.blur(img, (7, 7))         # smoothing
    thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
    kernel = np.ones((3, 3), np.uint8)
    img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
    img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
    contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny

    # Select the region of interest i.e the card and crop the background 
    contour1 = contours[1]
    x, y, w, h = cv2.boundingRect(contour1) # Find the bounding rectangle of contour 1

    crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle

    # Checking for color using HSV ranges

    # Convert the image to HSV
    hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

    # Define the range of colors for the Uno card

    # Green color range
    green_lower = np.array([36, 25, 25])
    green_upper = np.array([65, 255, 255])
    green_range = np.array([green_lower, green_upper])

    # Yellow color range
    y_lower = np.array([15, 50, 50])  
    y_upper = np.array([30, 255, 255])
    yellow_range = np.array([y_lower, y_upper])

    # Blue color range
    blue_lower = np.array([100, 150, 150])
    blue_upper = np.array([130, 255, 255])
    blue_range = np.array([blue_lower, blue_upper])

    # Red color range (two ranges to account for hue wraparound)
    red_lower1 = np.array([0, 50, 50])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([170, 50, 50])
    red_upper2 = np.array([180, 255, 255])
    red_range = np.array([red_lower1, red_upper1, red_lower2, red_upper2])

    # Create a mask for the red color
    mask_yellow = cv2.inRange(hsv, y_lower, y_upper)
    mask_red = cv2.inRange(hsv, red_lower1, red_upper1)
    mask_green = cv2.inRange(hsv, green_lower, green_upper)
    mask_blue = cv2.inRange(hsv, blue_lower, blue_upper)
    # Bitwise-AND mask and original image
    res_y = cv2.bitwise_and(crop_img,crop_img, mask= mask_yellow)
    res_r = cv2.bitwise_and(crop_img,crop_img, mask= mask_red)
    res_g = cv2.bitwise_and(crop_img,crop_img, mask= mask_green)
    res_b = cv2.bitwise_and(crop_img,crop_img, mask= mask_blue)

    # Check if the card is red, yellow , blue or green
    counts = [np.count_nonzero(res_r), np.count_nonzero(res_y), np.count_nonzero(res_g), np.count_nonzero(res_b)]
    colors = ["RED", "YELLOW", "GREEN", "BLUE"]

    if max(counts) == 0:
        det_color = "Not Recognised"
    else:
        color_index = counts.index(max(counts))
        det_color = colors[color_index]
    return det_color    

vc = cv2.VideoCapture(1) 
while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    feat = []
    try:
        fx = feature_extract(frame)
        col = color_extract(frame)
        feat.append(fx)
        feat = np.array(feat)
        rfc = joblib.load("uno_rfc.joblib") 
        predict = rfc.predict(feat)
        if predict == np.array([100]): # Pick two card is labeled as 100 in the dataset
            predict = 'Pick Two'
        elif predict == np.array([200]): # Reverse card is labeled as 100 in the dataset
            predict = 'Reverse'
        elif predict == np.array([300]): # Skip card is labeled as 100 in the dataset
            predict = 'Skip'
        card_type = [col, predict]
        cv2.putText(frame, str(card_type), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    except:
        pass
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 27:              # exit on ESC
        break
    
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()

## Appendix 

#### Scripts used for capturing images

For number cards

In [5]:
import cv2
vc = cv2.VideoCapture(1)
img_count = 0
special_cards = ['skip', 'reverse','picktwo']
while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 32:
        img_name = "./unocards/g{}.jpg".format(img_count)
        cv2.imwrite(img_name, frame)
        img_count += 1
    elif key == 27:              # exit on ESC
        break
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()   

For Pick Two, Reverse and Skip Cards

In [ ]:
import cv2
vc = cv2.VideoCapture(1)
img_count = 0
i = 0
special_cards = ['skip', 'reverse','picktwo']
wild_cards = ['pickfour', 'colorswitch']

while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 32:
            img_name = "./unocards/{}.jpg".format(wild_cards[i])
            i+=1
            cv2.imwrite(img_name, frame)
            
    elif key == 27:              # exit on ESC
        break
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()   